In [1]:
import pandas as pd
import numpy as np

In [2]:
ds = pd.read_csv("../FinalDatasets/CPMdataset.csv")

In [3]:
wd = pd.read_excel('../DataFiles/AnnualWeatherR2.xlsx')
wds = pd.read_excel('../DataFiles/AnnualWeatherR.xlsx')
wds = wds[['Year', 'FTCycles', 'Airport']]
wd = pd.concat([wds, wd], axis = 0)

In [4]:
wd.head()

,Year,FTCycles,Airport
0,1986,4,KBFI
1,1987,5,KBFI
2,1984,9,KBFI
3,1985,16,KBFI
4,1982,8,KBFI


In [5]:
def getCode(row):
    index = row['IntID']
    return ld.loc[index,'TargetID']

# Adding traffic data

In [6]:
dc = ds.drop_duplicates(subset = [u'UID'])
df = dc[['UID', 'Federal Functional Class', 'ESALs']]

In [7]:
df.columns = ['UID', 'Functional_Class', '2013']

In [8]:
def getClass(row):
    f = row['Functional_Class']
    req = f.split()[0]
    return req

In [9]:
df.loc[:,'Class'] = df.apply(getClass, axis = 1)

/Users/Alan/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pandas/core/indexing.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Alan/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
df.head()

,UID,Functional_Class,2013,Class
0,222,Urban Other Principal Arterial,134794,Urban
3,540,Rural Other Principal Arterial,52268,Rural
4,2431,Rural Other Principal Arterial,86888,Rural
5,704,Rural Other Freeway/Expressway,42668,Rural
8,776,Rural Other Principal Arterial,52542,Rural


In [11]:
de = pd.read_excel("../DataFiles/EsalIncrements.xlsx") 

In [12]:
de.head()

,Year,Rural,Urban,Total,ruralTMT,urbanTMT,totalTMT,rESALs,uESALs,tESALs,pRural,pUrban,pTotal
0,1980,6598173.2,8491214.5,15089387.6,1095738.317765,700176.161462,1691903.176381,739952.237659,419523.945046,1096669.995277,NaN,NaN,NaN
1,1981,7028297.3,9128681.7,16156978.9,1167167.703366,752741.002116,1811607.246514,788188.511643,451019.175155,1174260.642225,0.065188,0.075074,0.070751
2,1982,7112799.6,9243787.1,16356586.7,1181200.740275,762232.466162,1833988.345059,797665.023410,456706.167458,1188767.783991,0.012023,0.012609,0.012354
3,1983,7471215.6,9789398.8,17260614.4,1240721.782387,807223.003824,1935352.785930,837859.591387,483663.108994,1254470.917861,0.050390,0.059025,0.055270
4,1984,7417125.2,10614739.1,18031864.3,1231739.156120,875279.652629,2021829.466773,831793.622626,524440.552393,1310523.996131,-0.007240,0.084310,0.044683


In [17]:
di = de[['Year', 'pRural', 'pUrban']]
di.columns = ['Year', 'Rural', 'Urban']

In [14]:
def extrapolate(df):   
    for row in df.index:
        if df.loc[row, 'Class'] == 'Rural':
            cat = 'r'
        else:
            cat = 'u'
        for year in (np.arange(32)+1981)[::-1]:
            name = str(year)
            pname = str(year-1)
            ind  = year + 1 - 1980
            df.loc[row, name] = (df.loc[row, pname])/(1 + di.loc[ind, cat])

In [19]:
di.index = np.array(di.Year)

In [20]:
di.head()

,Year,Rural,Urban
1980,1980,NaN,NaN
1981,1981,0.065188,0.075074
1982,1982,0.012023,0.012609
1983,1983,0.050390,0.059025
1984,1984,-0.007240,0.084310


In [21]:
def extrapolate(row, year): 
    funClass = row['Class']
#     year  = 2013
    esal  = row[str(year)]
    r     = di.loc[year, funClass]
    required = esal/(1+r)
    return required

In [27]:
# extrapolate(df)
for year in (np.arange(32)+1981)[::-1]:
    name = str(year)
    pYear = year+1
    df.loc[:, name] = df.apply(extrapolate, axis =1, args = (pYear,))

In [26]:
(np.arange(32)+1981)[::-1]

array([2012, 2011, 2010, 2009, 2008, 2007, 2006, 2005, 2004, 2003, 2002,
       2001, 2000, 1999, 1998, 1997, 1996, 1995, 1994, 1993, 1992, 1991,
       1990, 1989, 1988, 1987, 1986, 1985, 1984, 1983, 1982, 1981])

In [28]:
df.head()

,UID,Functional_Class,2013,Class,2012,2011,2010,2009,2008,2007,...,1990,1989,1988,1987,1986,1985,1984,1983,1982,1981
0,222,Urban Other Principal Arterial,134794,Urban,126462.406367,127350.026602,128238.178975,127289.303357,125141.073011,129271.176692,...,85601.370140,83800.407148,80705.085746,77376.686137,74150.370174,69852.804663,67244.080099,62015.572011,58559.136905,57829.947364
3,540,Rural Other Principal Arterial,52268,Rural,56705.292107,57214.726903,58064.483748,57264.129970,55375.421978,58281.371710,...,48438.879274,46017.534794,44894.655482,42689.322039,40386.353710,37487.475310,37380.769806,37653.374182,35847.032017,35421.157984
4,2431,Rural Other Principal Arterial,86888,Rural,94264.357171,95111.218933,96523.816942,95193.344396,92053.640177,96884.361849,...,80522.639901,76497.504460,74630.879802,70964.831509,67136.479321,62317.512718,62140.130230,62593.295629,59590.512703,58882.558639
5,704,Rural Other Freeway/Expressway,42668,Rural,46290.300062,46706.167588,47399.850627,46746.496854,45204.685562,47576.903040,...,39542.169221,37565.550137,36648.908703,34848.626172,32968.641236,30602.196306,30515.089272,30737.624734,29263.051238,28915.396971
8,776,Rural Other Principal Arterial,52542,Rural,57002.553339,57514.658700,58368.870152,57564.320749,55665.711746,58586.895086,...,48692.806207,46258.768522,45130.002838,42913.108567,40598.067587,37683.992649,37576.727771,37850.761198,36034.949802,35606.843246


In [29]:
df2 = df[[u'2013',u'2012', u'2011', u'2010', u'2009', u'2008', u'2007', u'2006', u'2005', u'2004', u'2003', u'2002',
          u'2001', u'2000', u'1999',u'1998', u'1997', u'1996', u'1995', u'1994', u'1993', u'1992', u'1991',  u'1990',
          u'1989', u'1988', u'1987',u'1986', u'1985', u'1984', u'1983',u'1982', u'1981']]

In [30]:
df2.index = np.array(df.loc[:, 'UID'])

In [91]:
df2.head()

,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,...,1990,1989,1988,1987,1986,1985,1984,1983,1982,1981
222,134794,126462.406367,127350.026602,128238.178975,127289.303357,125141.073011,129271.176692,129023.163153,128826.808780,127988.456893,...,85601.370140,83800.407148,80705.085746,77376.686137,74150.370174,69852.804663,67244.080099,62015.572011,58559.136905,57829.947364
540,52268,56705.292107,57214.726903,58064.483748,57264.129970,55375.421978,58281.371710,57439.885668,56913.670884,57220.694521,...,48438.879274,46017.534794,44894.655482,42689.322039,40386.353710,37487.475310,37380.769806,37653.374182,35847.032017,35421.157984
2431,86888,94264.357171,95111.218933,96523.816942,95193.344396,92053.640177,96884.361849,95485.512855,94610.756787,95121.139236,...,80522.639901,76497.504460,74630.879802,70964.831509,67136.479321,62317.512718,62140.130230,62593.295629,59590.512703,58882.558639
704,42668,46290.300062,46706.167588,47399.850627,46746.496854,45204.685562,47576.903040,46889.971717,46460.406162,46711.039141,...,39542.169221,37565.550137,36648.908703,34848.626172,32968.641236,30602.196306,30515.089272,30737.624734,29263.051238,28915.396971
776,52542,57002.553339,57514.658700,58368.870152,57564.320749,55665.711746,58586.895086,57740.997795,57212.024481,57520.657602,...,48692.806207,46258.768522,45130.002838,42913.108567,40598.067587,37683.992649,37576.727771,37850.761198,36034.949802,35606.843246


In [131]:
df2_reordered = df2[df2.columns[::-1]]
df2_reordered.head()

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
222,57829.947364,58559.136905,62015.572011,67244.080099,69852.804663,74150.370174,77376.686137,80705.085746,83800.407148,85601.370140,...,127988.456893,128826.808780,129023.163153,129271.176692,125141.073011,127289.303357,128238.178975,127350.026602,126462.406367,134794
540,35421.157984,35847.032017,37653.374182,37380.769806,37487.475310,40386.353710,42689.322039,44894.655482,46017.534794,48438.879274,...,57220.694521,56913.670884,57439.885668,58281.371710,55375.421978,57264.129970,58064.483748,57214.726903,56705.292107,52268
2431,58882.558639,59590.512703,62593.295629,62140.130230,62317.512718,67136.479321,70964.831509,74630.879802,76497.504460,80522.639901,...,95121.139236,94610.756787,95485.512855,96884.361849,92053.640177,95193.344396,96523.816942,95111.218933,94264.357171,86888
704,28915.396971,29263.051238,30737.624734,30515.089272,30602.196306,32968.641236,34848.626172,36648.908703,37565.550137,39542.169221,...,46711.039141,46460.406162,46889.971717,47576.903040,45204.685562,46746.496854,47399.850627,46706.167588,46290.300062,42668
776,35606.843246,36034.949802,37850.761198,37576.727771,37683.992649,40598.067587,42913.108567,45130.002838,46258.768522,48692.806207,...,57520.657602,57212.024481,57740.997795,58586.895086,55665.711746,57564.320749,58368.870152,57514.658700,57002.553339,52542


In [132]:
df2_cum_sum = df2_reordered.cumsum(axis=1)
df2_cum_sum.head()

,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,...,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
222,57829.947364,116389.084269,178404.656280,245648.736379,315501.541042,389651.911216,467028.597353,547733.683099,631534.090247,717135.460388,...,2226489.331504,2355316.140285,2484339.303437,2613610.480129,2738751.553140,2866040.856497,2994279.035472,3121629.062074,3248091.468441,3382885.468441
540,35421.157984,71268.190002,108921.564184,146302.333990,183789.809300,224176.163010,266865.485049,311760.140531,357777.675324,406216.554598,...,1215873.022028,1272786.692911,1330226.578580,1388507.950289,1443883.372267,1501147.502237,1559211.985985,1616426.712889,1673132.004996,1725400.004996
2431,58882.558639,118473.071341,181066.366971,243206.497201,305524.009919,372660.489241,443625.320750,518256.200552,594753.705012,675276.344914,...,2021213.268882,2115824.025669,2211309.538525,2308193.900374,2400247.540551,2495440.884947,2591964.701888,2687075.920821,2781340.277992,2868228.277992
704,28915.396971,58178.448209,88916.072943,119431.162215,150033.358522,183001.999757,217850.625929,254499.534633,292065.084770,331607.253991,...,992555.102622,1039015.508784,1085905.480501,1133482.383541,1178687.069103,1225433.565957,1272833.416584,1319539.584173,1365829.884235,1408497.884235
776,35606.843246,71641.793049,109492.554246,147069.282017,184753.274666,225351.342253,268264.450820,313394.453658,359653.222180,408346.028387,...,1222246.887644,1279458.912125,1337199.909920,1395786.805007,1451452.516753,1509016.837502,1567385.707654,1624900.366354,1681902.919693,1734444.919693


In [133]:
df2_cum_sum['id'] = df2_cum_sum.index

In [134]:
desired_table = df2_cum_sum.stack()
desired_table.head()

222  1981     57829.947364
     1982    116389.084269
     1983    178404.656280
     1984    245648.736379
     1985    315501.541042
dtype: float64

In [137]:
desired_table.head(50)

222  1981      57829.947364
     1982     116389.084269
     1983     178404.656280
     1984     245648.736379
     1985     315501.541042
     1986     389651.911216
     1987     467028.597353
     1988     547733.683099
     1989     631534.090247
     1990     717135.460388
     1991     806392.460607
     1992     900622.831066
     1993     996976.133792
     1994    1095759.749885
     1995    1196450.959860
     1996    1299003.213072
     1997    1406182.935623
     1998    1516512.366582
     1999    1630013.344499
     2000    1745073.508649
     2001    1860827.344050
     2002    1979635.373579
     2003    2098500.874611
     2004    2226489.331504
     2005    2355316.140285
     2006    2484339.303437
     2007    2613610.480129
     2008    2738751.553140
     2009    2866040.856497
     2010    2994279.035472
     2011    3121629.062074
     2012    3248091.468441
     2013    3382885.468441
     id          222.000000
540  1981      35421.157984
     1982      71268

In [31]:
def fillESALs(row):
    index = row['UID']
    value = str(row['sYear'])
    return df2.loc[index, value]

In [32]:
sds = ds[ds.sYear < 2014]

In [33]:
#"Extrapolated" ESALs..
sds.loc[:,'ESALs'] = sds.apply(fillESALs, axis = 1)

In [34]:
#Exporting the almost complete dataset
sds.to_csv("../FinalDatasets/CPMdatasetWesals.csv") 

In [35]:
sds.head()

,Unnamed: 0,sYear,Rutting,IRI,PSC,IntID,Alli,Long,Low Flexible Patching,Medium Flexible Patching,...,Total Surface Thickness,Listed Surface Type,Pavement Type,Construction Pavement Type,Major Rehab Type,Prior Major Rehab Year,Prior Major Rehab Type,AlliP,LongP,STcode
0,0,1981,0.236,190.039,92,4,0,46,0,0,...,0.15,ACP,Composite,PCCP,ACP,1989,ACP,0.000000,3.960055,PA
1,1,1981,0.236,190.039,92,4,0,46,0,0,...,0.15,ACP,Composite,PCCP,ACP,1989,ACP,0.000000,3.960055,BA
2,2,1981,0.236,190.039,92,4,0,46,0,0,...,0.15,ACP,Composite,PCCP,ACP,1989,ACP,0.000000,3.960055,Other
3,3,1981,0.236,226.083,83,12,0,0,0,16,...,0.90,ACP,ACP,Unknown,ACP,1984,ACP,0.000000,0.000000,Other
4,4,1981,0.512,245.150,37,24,63,158,52,0,...,0.98,ACP,Composite,PCCP,ACP,1987,ACP,5.965909,14.962121,BA


# Adding coordinates to dataset to identify segment locations

In [37]:
milePost = pd.read_csv("../FinalDatasets/mileposts.csv")

In [38]:
milePost.head()

,MP_VALUE,SR,LONGITUDE,LATITUDE,REGION
0,1,2,-122.169366,47.978215,NW
1,1,2,-122.167960,47.978424,NW
2,2,2,-122.147555,47.978414,NW
3,3,2,-122.133338,47.970205,NW
4,3,2,-122.132785,47.970380,NW


In [49]:
routeNames = np.unique(np.array(milePost.SR))
print routeNames

[  2   3   4   5   6   7   8   9  10  11  12  14  16  17  18  19  20  21
  22  23  24  25  26  27  28  31  41  82  90  92  96  97  99 100 101 102
 103 104 105 106 107 108 109 110 112 113 115 116 117 119 121 122 123 124
 125 127 128 129 131 141 142 150 153 155 160 161 162 163 164 165 166 167
 169 170 172 173 174 181 182 193 194 195 197 202 203 204 205 206 207 211
 215 221 223 224 225 231 240 241 243 260 261 262 263 270 271 272 274 278
 281 282 283 290 291 292 300 302 303 305 307 308 310 395 397 401 405 409
 410 411 432 500 501 502 503 504 505 506 507 508 509 510 512 515 516 518
 520 522 524 525 526 527 529 530 531 532 534 536 538 539 542 544 546 547
 548 702 705 706 730 821 823 900 902 903 904 908 970 971]


In [50]:
# for name in routeNames:
#     if len(name) > 3:
#         print int(float(re.findall("[-+]?\d+[\.]?\d*", name)[0]))

In [47]:
import re
def change2ints(row):
    sr = row['SR']
    if len(sr) > 3:
        return int(float(re.findall("[-+]?\d+[\.]?\d*", name)[0]))
    return int(sr)

In [48]:
milePost.loc[:,'SR'] = milePost.apply(change2ints, axis = 1)

In [53]:
db = milePost[['LONGITUDE', 'LATITUDE', 'MP_VALUE', 'SR']] 

In [54]:
db.head()

,LONGITUDE,LATITUDE,MP_VALUE,SR
0,-122.169366,47.978215,1,2
1,-122.167960,47.978424,1,2
2,-122.147555,47.978414,2,2
3,-122.133338,47.970205,3,2
4,-122.132785,47.970380,3,2


In [59]:
print int(1.98)
sr = 2
bmp = 0.99
print np.array(db[((db.loc[:,'MP_VALUE'] > bmp) & (db.loc[:,'MP_VALUE'] < (bmp+2))) & (db.loc[:,'SR'] == sr)])

1
[[-122.169366   47.978215    1.          2.      ]
 [-122.16796    47.978424    1.          2.      ]
 [-122.147555   47.978414    2.          2.      ]]


In [76]:
def getLat(row):
    sr = row['SR']
    bmp = int(row['Begin ARM'])
#     emp = int(row['End ARM'])
    res = np.array(db[((db.loc[:,'MP_VALUE'] > (bmp-1)) & (db.loc[:,'MP_VALUE'] < (bmp+4))) & (db.loc[:,'SR'] == sr)])
    if len(res) > 0:
        coordinates = res[0][0:2]
        x = coordinates[0]
        return coordinates[1]
    return None
sds.loc[:,'Latitude'] = sds.apply(getLat, axis = 1)

In [77]:
def getLong(row):
    sr = row['SR']
    bmp = int(row['Begin ARM'])+1
#     emp = int(row['End ARM'])+1
    res = np.array(db[((db.loc[:,'MP_VALUE'] > (bmp-1)) & (db.loc[:,'MP_VALUE'] < (bmp+4))) & (db.loc[:,'SR'] == sr)])
    if len(res) > 0:
        coordinates = res[0][0:2]
        return coordinates[0]
#         return coordinates[1]
    return None
sds.loc[:,'Longitude'] = sds.apply(getLong, axis = 1)

In [65]:
sds.head()

,Unnamed: 0,sYear,Rutting,IRI,PSC,IntID,Alli,Long,Low Flexible Patching,Medium Flexible Patching,...,Pavement Type,Construction Pavement Type,Major Rehab Type,Prior Major Rehab Year,Prior Major Rehab Type,AlliP,LongP,STcode,Latitude,Longitude
0,0,1981,0.236,190.039,92,4,0,46,0,0,...,Composite,PCCP,ACP,1989,ACP,0.000000,3.960055,PA,47.863551,-121.781888
1,1,1981,0.236,190.039,92,4,0,46,0,0,...,Composite,PCCP,ACP,1989,ACP,0.000000,3.960055,BA,47.863551,-121.781888
2,2,1981,0.236,190.039,92,4,0,46,0,0,...,Composite,PCCP,ACP,1989,ACP,0.000000,3.960055,Other,47.863551,-121.781888
3,3,1981,0.236,226.083,83,12,0,0,0,16,...,ACP,Unknown,ACP,1984,ACP,0.000000,0.000000,Other,47.716004,-121.231507
4,4,1981,0.512,245.150,37,24,63,158,52,0,...,Composite,PCCP,ACP,1987,ACP,5.965909,14.962121,BA,47.642802,-118.279214


In [78]:
cdf = sds.drop_duplicates(subset = ['UID'])

In [84]:
cleaned = cdf[~np.isnan(cdf.Longitude)]
cleaned = cleaned[~np.isnan(cleaned.Latitude)]

In [85]:
print len(cdf)
print len(cleaned)

9585
8938


In [86]:
cleaned.index = range(len(cleaned))
coordata = cleaned[['UID', 'Longitude', 'Latitude']]

In [87]:
coordata.to_csv('../FinalDatasets/RCsegCoord.csv')

In [88]:
sds = sds[~np.isnan(sds.Longitude)]
sds = sds[~np.isnan(sds.Latitude)]

In [89]:
print len(sds)

427989


In [90]:
sds.to_csv('../FinalDatasets/RoughnessCrackingDataset.csv')

In [93]:
sds[['UID', 'cYear', 'Major Rehab Year', 'Prior Major Rehab Year']].head()

,UID,cYear,Major Rehab Year,Prior Major Rehab Year
0,222,1941,2000,1989
1,222,1989,2000,1989
2,222,2000,2000,1989
3,540,1999,1998,1984
4,2431,1987,1999,1987


In [95]:
print len(np.unique(np.array(sds[sds.cYear > 1970].UID)))

6875


In [96]:
df.head()

,UID,Functional_Class,2013,Class,2012,2011,2010,2009,2008,2007,...,1990,1989,1988,1987,1986,1985,1984,1983,1982,1981
0,222,Urban Other Principal Arterial,134794,Urban,126462.406367,127350.026602,128238.178975,127289.303357,125141.073011,129271.176692,...,85601.370140,83800.407148,80705.085746,77376.686137,74150.370174,69852.804663,67244.080099,62015.572011,58559.136905,57829.947364
3,540,Rural Other Principal Arterial,52268,Rural,56705.292107,57214.726903,58064.483748,57264.129970,55375.421978,58281.371710,...,48438.879274,46017.534794,44894.655482,42689.322039,40386.353710,37487.475310,37380.769806,37653.374182,35847.032017,35421.157984
4,2431,Rural Other Principal Arterial,86888,Rural,94264.357171,95111.218933,96523.816942,95193.344396,92053.640177,96884.361849,...,80522.639901,76497.504460,74630.879802,70964.831509,67136.479321,62317.512718,62140.130230,62593.295629,59590.512703,58882.558639
5,704,Rural Other Freeway/Expressway,42668,Rural,46290.300062,46706.167588,47399.850627,46746.496854,45204.685562,47576.903040,...,39542.169221,37565.550137,36648.908703,34848.626172,32968.641236,30602.196306,30515.089272,30737.624734,29263.051238,28915.396971
8,776,Rural Other Principal Arterial,52542,Rural,57002.553339,57514.658700,58368.870152,57564.320749,55665.711746,58586.895086,...,48692.806207,46258.768522,45130.002838,42913.108567,40598.067587,37683.992649,37576.727771,37850.761198,36034.949802,35606.843246


In [97]:
cumdf = df[['UID', u'1981']]

In [113]:
segments = np.unique(np.array(df.UID))
msegs = np.tile(segments, 33)
print len(msegs) == len(segments) *33

True


In [114]:
years = np.arange(33)+1981
myears = np.tile(years, len(segments))
print len(msegs) == len(myears)

True


In [119]:
sorted_segments = np.sort(msegs)
print sorted_segments[:40]
print msegs[0:40]
print myears[:40]

[  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
   1.   1.   1.  24.  24.  24.  24.  24.  24.  24.]
[   1.   24.   27.   28.   32.   33.   37.   38.   39.   43.   66.   81.
   97.  102.  104.  110.  113.  114.  115.  117.  118.  120.  125.  138.
  142.  143.  144.  150.  153.  154.  155.  159.  188.  218.  222.  226.
  232.  233.  235.  240.]


In [126]:
segDic = {'ID': sorted_segments, 'Year': myears}

In [127]:
cumdf = pd.DataFrame(segDic)

In [140]:
cumdf.head()

,ID,Year
0,1,1981
1,1,1982
2,1,1983
3,1,1984
4,1,1985


In [139]:
print len(cumdf)

316305


In [141]:
def getEsal(row, year):
    id = row['ID']
    name = str(year)
    return df2.loc[id, name]

In [ ]:
for year in years:
    cumdf.loc[:, str(year)] =cumdf.apply(getEsal, axis =1, args = (year,))